In [6]:
all_locations = {
    "1. Istanbul, Turkey: A city that straddles Europe and Asia across the Bosphorus Strait.": [41.0082, 28.9784],
    "2. Athens, Greece: The capital city known for its ancient landmarks.": [37.9838, 23.7275],
    "3. Vienna, Austria: Austria's capital, known for its artistic and intellectual legacy.": [48.2082, 16.3738],
    "4. Tirana, Albania: The capital city of Albania, known for its colorful Ottoman, Fascist and Soviet-era architecture.": [41.3275, 19.8187],
    "5. Rome, Italy: The capital city, known for its nearly 3,000 years of globally influential art, architecture, and culture.": [41.9028, 12.4964],
    "6. Paris, France: The capital city, known for its cafe culture and landmarks like the Eiffel Tower.": [48.8566, 2.3522],
    "7. Stockholm, Sweden: The capital city, encompassing 14 islands and more than 50 bridges.": [59.3293, 18.0686],
    "8. Reykjavik, Iceland: The capital city, known for its modernist architecture and vibrant cultural scene.": [64.1265, -21.8174],
}

countries_to_highlight = ["China", "Mexico", "United States", "Canada", "Guatemala", "Honduras", "Spain", "United Kingdom", "Austria", "Italy", "France", "Sweden", "Iceland", "Greece", "Albania", "Türkiye"]


SyntaxError: invalid syntax (1145881997.py, line 12)

In [4]:
import folium
import geopandas as gpd
import pycountry
from folium import IFrame

def create_auto_centered_map(locations, output_file="map_route.html"):
    # Initialize map without specifying center and zoom
    auto_center_map = folium.Map()

    # Add all locations to the map
    for location, coordinates in locations.items():
        # Split location name and description
        title, description = location.split(": ", 1)
        
        # Generate link to corresponding .md file
        # Create HTML content for popup with short description and link
        html_content = f"""
        <div style="font-family: Arial, sans-serif; font-size: 12px;">
            <b>{title}</b><br>
            {description.split('.')[0]}.<br>
        </div>
        """
        iframe = IFrame(html_content, width=250, height=150)  # Adjust popup size

        # Add marker with formatted popup
        folium.Marker(
            location=coordinates,
            popup=folium.Popup(iframe),
        ).add_to(auto_center_map)

    # Add edges from i to i+1
    locations_list = list(locations.values())
    for i in range(len(locations_list) - 1):
        folium.PolyLine(locations_list[i:i+2], color="blue", weight=2.5, opacity=1).add_to(auto_center_map)

    # Save map to HTML file
    auto_center_map.save(output_file)
    print(f"Map has been saved to {output_file}")

def highlight_countries_on_map(countries, shapefile_path, output_file="highlighted_countries_map.html"):
    # Load the world shapefile from the local file
    world = gpd.read_file(shapefile_path)

    # Convert country names to ISO Alpha-3 codes using pycountry
    country_codes = []
    for country in countries:
        try:
            country_code = pycountry.countries.lookup(country).alpha_3
            country_codes.append(country_code)
        except LookupError:
            print(f"Warning: '{country}' not found in pycountry database.")

    # Filter for the specified countries by ISO code
    highlighted = world[world["ADM0_A3"].isin(country_codes)]

    # Initialize a folium map centered globally
    world_map = folium.Map(location=[20, 0], zoom_start=2)

    # Add highlighted countries to the map in red
    folium.GeoJson(
        highlighted,
        style_function=lambda x: {'fillColor': 'red', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.7}
    ).add_to(world_map)

    # Save the map to an HTML file
    world_map.save(output_file)
    print(f"Map has been saved to {output_file}")

create_auto_centered_map(all_locations)

# List of countries to highlight

# Call the function to highlight countries and save to HTML
highlight_countries_on_map(countries_to_highlight, shapefile_path="/Users/songye03/Desktop/me/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

Map has been saved to map_route.html
Map has been saved to highlighted_countries_map.html


In [5]:
# Generate the HTML content
html_content = """
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Travel Locations</title>
    <style>
        body { font-family: Arial, sans-serif; line-height: 1.6; margin: 2em; }
        h1 { text-align: center; }
        ul { list-style-type: none; padding: 0; }
        li { margin-bottom: 1em; }
        a { text-decoration: none; color: #007BFF; }
        a:hover { text-decoration: underline; }
    </style>
</head>
<body>
    <h1>Travel Locations</h1>
    <ul>
"""

# Add each location with its description and hyperlink
for location in all_locations:
    title, description = location.split(": ", 1)
    # Update the city_name links to point to blog posts
    city_name = " ".join(title.split(". ", 1)[-1].split(",")[0].strip().split()).lower().replace(" ", "-")
    html_content += f"""
        <li>
            <strong>{title.split(". ", 1)[-1]}</strong>: {description}
            <br>
            <a href="/{city_name}/" target="_blank">Read more</a>
        </li>
    """

# Close the HTML tags
html_content += """
    </ul>
</body>
</html>
"""

# Save the HTML content to a file
output_file = "_includes/locations_list.html"
with open(output_file, "w", encoding="utf-8") as file:
    file.write(html_content)

print(f"HTML file has been created: {output_file}")

HTML file has been created: _includes/locations_list.html
